# Model

In [17]:
import math, random
from statistics import median

beta = [alpha, beta_1, ..., beta_k]

x_i = [1, x_i1, ..., x_ik]

In [6]:
def predict(x_i, beta):
    return dot(x_i, beta)

# Model selection

In [7]:
def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)


def squared_error(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2


def squared_error_gradient(x_i, y_i, beta):
    # the gradient corresponding to the ith squared error term
    return [-2 * x_ij * error(x_i, y_i, beta)
            for x_ij in x_i]

In [9]:
def estimate_beta(x, y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error, 
                               squared_error_gradient, 
                               x, y, 
                               beta_initial, 
                               0.001)

# Quality of model selection

In [10]:
def multiple_r_squared(x, y, beta):
    sum_of_squared_errors = sum(error(x_i, y_i, beta) ** 2
                                for x_i, y_i in zip(x, y))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(y)

# Retreat: data bootstrapping

data = get_sample(num_points = n)

In [12]:
def bootstrap_sample(data):
    # randomly samples len(data) elements with replacement
    return [random.choice(data) for _ in data]


def bootstrap_statistic(data, stats_fn, num_samples):
    # evaluates stats_fn on num_samples bootstrap samples from data
    return [stats_fn(bootstrap_sample(data)) 
            for _ in range(num_samples)]

In [14]:
close_to_100 = [99.5 + random.random() for _ in range(101)]

far_from_100 = ([99.5 + random.random()] +
                [random.random() for _ in range(50)] + 
                [200 + random.random() for _ in range(50)])

In [18]:
bootstrap_statistic(close_to_100, median, 100)

[100.07565101416489,
 100.08980118353116,
 100.10318562796138,
 100.11277317986861,
 100.09628686158311,
 100.1127831050407,
 100.07565101416489,
 100.28379858903477,
 100.08338203945503,
 100.11277317986861,
 100.16024537862239,
 100.15665938898962,
 100.07969501074561,
 100.10318562796138,
 100.08338203945503,
 100.13014734041147,
 100.1127831050407,
 100.08338203945503,
 100.09628686158311,
 100.08761706417543,
 100.15665938898962,
 100.07565101416489,
 100.1127831050407,
 100.06751074062068,
 100.08980118353116,
 100.07969501074561,
 100.13014734041147,
 100.16024537862239,
 100.11836899667533,
 100.1108869734438,
 100.08338203945503,
 100.08980118353116,
 100.15665938898962,
 100.07565101416489,
 100.09628686158311,
 100.10318562796138,
 100.07565101416489,
 100.05126724609055,
 100.09628686158311,
 100.06751074062068,
 100.11836899667533,
 100.1127831050407,
 100.08980118353116,
 100.04869930383559,
 100.10318562796138,
 100.13014734041147,
 100.09628686158311,
 100.0898011835311

In [19]:
bootstrap_statistic(far_from_100, median, 100)

[200.0580509438265,
 100.34507757567151,
 200.0580509438265,
 100.34507757567151,
 0.9882351487225011,
 200.0467796859568,
 200.25922692344722,
 200.0456964935684,
 0.9610312802396112,
 200.0456964935684,
 200.23941596018597,
 0.9369691586445807,
 0.9184820619953314,
 100.34507757567151,
 0.8383265651934163,
 0.8007465903541809,
 0.7315983062253606,
 100.34507757567151,
 200.25922692344722,
 0.9805166506472687,
 100.34507757567151,
 0.9882351487225011,
 200.19230954124467,
 200.01243631882932,
 0.9805166506472687,
 0.8007465903541809,
 100.34507757567151,
 0.9665489030431832,
 200.24013040782634,
 0.9665489030431832,
 200.17481948445143,
 0.9610312802396112,
 200.01243631882932,
 200.00152422185673,
 0.9369691586445807,
 0.9184820619953314,
 200.24013040782634,
 0.9805166506472687,
 0.9100160146990397,
 200.23941596018597,
 0.9805166506472687,
 200.23941596018597,
 0.9610312802396112,
 0.8383265651934163,
 200.01243631882932,
 0.9805166506472687,
 200.0467796859568,
 200.23941596018597

# Regression coefficients standart errors

In [20]:
def estimate_sample_beta(sample):
    x_sample, y_sample = zip(*sample) # magic unzipping trick
    return estimate_beta(x_sample, y_sample)

bootstrap_standart_errors = [
    standart_deviation([beta[i] for beta in boottrap_betas])
    for i in range(4)]

In [24]:
def p_value(beta_hat_j, sigma_hat_j):
    if beta_hat_j > 0:
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)

# Regularization

In [26]:
# alpha is a hyperparameter controlling how harsh the penalty is
# sometimes it's called lambda but that already means something in Python
def ridge_penalty(beta, alpha):
  return alpha * dot(beta[1:], beta[1:])


def squared_error_ridge(x_i, y_i, beta, alpha):
    # estimate error plus ridge penalty on beta
    return error(x_i, y_i, beta) ** 2 + ridge_penalty(beta, alpha)


def ridge_penalty_gradient(beta, alpha):
    # gradient of just the ridge penalty
    return [0] + [2 * alpha * beta_j for beta_j in beta[1:]]


def squared_error_ridge_gradient(x_i, y_i, beta, alpha):
    # the gradient corresponding to the ith squared error term
    # including the ridge penalty
    return vector_add(squared_error_gradient(x_i, y_i, beta),
                      ridge_penalty_gradient(beta, alpha))


def estimate_beta_ridge(x, y, alpha):
    # use gradient descent to fit a ridge regression
    # with penalty alpha
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(partial(squared_error_ridge, alpha=alpha), 
                               partial(squared_error_ridge_gradient, 
                                       alpha=alpha), 
                               x, y, 
                               beta_initial, 
                               0.001)

In [27]:
def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])    